In [28]:
import numpy as np
import pandas as pd
import fasttext
import os
import os.path

In [29]:
dirs = os.getcwd()
len(os.listdir(dirs))

354

In [30]:
'''
 *各种字符的unicode编码的范围：
     * 汉字：[0x4e00,0x9fa5]（或十进制[19968,40869]）
     * 数字：[0x30,0x39]（或十进制[48, 57]）
     *小写字母：[0x61,0x7a]（或十进制[97, 122]）
     * 大写字母：[0x41,0x5a]（或十进制[65, 90]）
'''
import re
zhPattern = re.compile(u'[\u4e00-\u9fa5]+')
numPattern = re.compile('[0-9]')
def deal(x):
    if x == -888:  #这是缺失值填充的（记得先填充缺失值）
        return x
    contents = str(x)
    zhmatch = zhPattern.search(contents)
    if zhmatch:  
#         print (u'有中文：%s' % (zhmatch.group(0),))  #记住是怎么打印的
        return -666  #汉字 直接返回
    numbermatch = numPattern.search(contents)
    if numbermatch:
        return 8 #表示 数字 
    else:
#         print (u'有字母：%s' % (x))  #记住是怎么打印的
        return 9 #表示字母
        

In [31]:
def prepross(data):
    #1 使用填充的方式把
    dataPro = data.copy()
    label = dataPro.columns
    #统计每行的缺失率
    dataPro.insert(0,"missingRate",0)
    dataPro["missingRate"] = dataPro.isnull().sum(axis = 1)/dataPro.shape[1]
    
    dataPro.fillna(-888,inplace = True)
    # 数字和空缺值填充，保留分布格局
    for i in range(len(label)):
        dataPro[label[i]] = dataPro[label[i]].apply(lambda x: deal(x) )
        
    dataPro.insert(1,"index",dataPro.index)
    dataPro.insert(2,"position",0)
    dataPro.insert(3,"first_last_No",0)
    dataPro.insert(4,"class2_number",0)
    dataPro.insert(5,"alphabet",0)
    
    number1 = 0
    for i in range(dataPro.shape[0]):
        if i < int(dataPro.shape[0]/3):
            dataPro["position"][i] = 1
        elif i < int(2*dataPro.shape[0]/3) and i >= int(dataPro.shape[0]/3):
            dataPro["position"][i] = 2
        else:
            dataPro["position"][i] = 3

        # 下面表示的是 头部加上序列号（2标签可能全是数字），尾部全部用 -1
        #  (dataPro.iloc[i,:] == -666).sum() 比较保险，data[i] 出错。
        if (dataPro.iloc[i,:] == 8).sum() < (dataPro.iloc[i,:] == -666).sum() and number1 == 0:
                dataPro["first_last_No"][i] = i
        else:
             #针对 class2 全是数字的情况
            if i < int(dataPro.shape[0]/3):
                if (dataPro.iloc[i+1,:] == 8).sum() < (dataPro.iloc[i+1,:] == -666).sum()and number1 == 0: 
                    dataPro["first_last_No"][i] = i  #仍然编码 ，原来是i
                    dataPro["class2_number"][i] = 1 
                else:    
                    number1 += 1
        #中间class3 不用管 就是0
        if (dataPro.iloc[i,:] == 8).sum() < (dataPro.iloc[i,:] == -666).sum() and number1 > 0:
               dataPro["first_last_No"][i] = -1 
        # 对字母比较多的行进行标记
        if (dataPro.iloc[i,:] == 9).sum() > (dataPro.iloc[i,:] == -666).sum() and i < int(dataPro.shape[0]/3):
            dataPro["alphabet"][i] = 1
    return dataPro

In [32]:
number = 0
fulldata = 0
kk = 0
for path,dirname,filenames in os.walk(dirs): 
    for filename in filenames:
        if filename.endswith('.csv'):  #因为有ipython 在所以必须有这句
            number += 1
            data = pd.read_csv(filename,encoding = "gb18030",header = None)
            cc = prepross(data.iloc[:,1:])
#             cc = pd.Series(cc)
            dataok = pd.concat([data.iloc[:,0],cc],axis = 1,ignore_index = True)
#             dataok.columns = ["Y","content"]
            if kk == 0:
                fulldata = dataok
                kk = 5
            fulldata = pd.concat([fulldata,dataok],axis = 0,ignore_index = True)
            if number % 10 == 0:
                print(number)
#             print(1,data.head())
#上面多了 ipython 和目录

d:\gongju\python3.6 64\anzhuang\lib\site-packages\ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
d:\gongju\python3.6 64\anzhuang\lib\site-packages\ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
d:\gongju\python3.6 64\anzhuang\lib\site-packages\ipykernel_launcher.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
d:\gongju\python3.6 64\anzhuang\lib\site-packages\ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set o

10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350


In [33]:
fulldata.to_csv("../trainFullNumberNew88.csv",header = None,index = False)

In [34]:
# fulldata = pd.DataFrame(fulldata)
# label = fulldata.columns
# for i in range(fulldata.shape[0]):
#     print(fulldata[label[i].unique()])

In [35]:
# # 测试
# data = pd.read_csv("add2007nianjianhead1.csv",encoding = "gb18030",header = None)
# (data[0] == 3).sum()
# # # print("data",data.shape)
# # a = [[1,3,2],[4,6,2]]
# # (np.array(a[0]) > 1).sum()  #list很多功能都没有
# # print("2222....................")
# cc = prepross(data.iloc[:,1:])
# # cc.to_csv("add2007nianjianhead7mm.csv",encoding = "gb18030",header = None)
# cc
# # cc = pd.Series(cc)
# dataok = pd.concat([data.iloc[:,0],cc],axis = 1)
# dataok.columns = ["Y","content"]
# # len(dataok) #回来就不是  pandas

In [36]:
# mystring = u'今年18岁'
# mystring2 = mystring.encode('gbk')
# print(type(mystring2))
# print(filter(str.isdigit, mystring2))
# mm = "wo 很 开心 18"
# if mm  >= u'\u4e00' and mm <= u'\u9fa5':
#     print("包含汉字") 
# else:
#     print("bu包含汉字") 

# -*- coding: utf-8 -*-
# import re
# zhPattern = re.compile(u'[\u4e00-\u9fa5]+')
# #一个小应用，判断一段文本中是否包含简体中：
# contents=u'一个小应用，判断一段文本中是否包含简体中：'
# contents = '0.26'
# match = zhPattern.search(contents)
# if match:
#     print (u'有中文：%s' % (match.group(0),))
# else:
#     print (u'没有包含中文')